<a href="https://colab.research.google.com/github/elorie-bernard-lacroix/SmartStudy/blob/main/SmartStudy/notebooks/3_41_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Final Model Version Complete

In [1]:
!pip install tabpfn
!pip install bayesian-optimization
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install openai
!pip install gradio
!pip install scikit-optimize



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

# Load & Preprocess Data


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor

import openai
import getpass

from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/ECE324_Project/Model/database.csv") #load the dataset
data = data.interpolate(method='linear', limit_direction='forward')
data.duplicated().sum()
data.drop_duplicates(inplace=True)

columns_to_keep = ['Age', 'Gender', 'ParentalEducation', # select relevant columns
                   'StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport',
                   'Extracurricular', 'Sports', 'Music', 'Volunteering', 'GPA']
data = data[columns_to_keep]

scaler = StandardScaler()
input = scaler.fit_transform(data.drop('GPA', axis=1))

labels = data['GPA']

X_train, X_test, Y_train, Y_test = train_test_split(input, labels, test_size=0.2, random_state=42)

# Train the Model using TabPFN

In [5]:
reg = TabPFNRegressor(random_state=42)
reg.fit(X_train, Y_train)

Y_pred = reg.predict(X_test)
print("MSE:", mean_squared_error(Y_test, Y_pred))
print("MAE:", mean_absolute_error(Y_test, Y_pred))

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:101: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-regressor.ckpt.
  model, bardist, config_ = load_model_criterion_config(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

MSE: 0.039094818963556204
MAE: 0.15616286595164913


## Optimize Habits

In [17]:
# user input
age = 15
gender = 0
study_time_weekly = 4.2
absences = 10
extracurricular = 1
sports = 0
music = 0
volunteering = 0
parental_education = 1
parental_support = 1
tutoring = 1
desired_grade = 4.0


space = [
    Real(0.0, 20.0, name='StudyTimeWeekly'),  # Continuous variable
    Integer(0, 29, name='Absences'),  # Integer variable
    Categorical([0, 1], name='Tutoring'),  # Categorical variable
    Integer(0, 4, name='ParentalSupport'),  # Integer variable
    Categorical([0, 1], name='Extracurricular'),  # Categorical variable
    Categorical([0, 1], name='Sports'),  # Categorical variable
    Categorical([0, 1], name='Music'),  # Categorical variable
    Categorical([0, 1], name='Volunteering')  # Categorical variable
  ]

@use_named_args(space)
def objective(**params):
    user_data = {
        'Age': age,
        'Gender': gender,
        'ParentalEducation': parental_education,
        'StudyTimeWeekly': params['StudyTimeWeekly'],
        'Absences': params['Absences'],
        'Tutoring': params['Tutoring'],
        'ParentalSupport': params['ParentalSupport'],
        'Extracurricular': params['Extracurricular'],
        'Sports': params['Sports'],
        'Music': params['Music'],
        'Volunteering': params['Volunteering']
    }

    user_df = pd.DataFrame(user_data, index=[0])
    print(user_df)

    #process data
    user_input = scaler.transform(user_df)

    # predict grade
    pred_grade = reg.predict([user_input[0]])
    print(pred_grade)
    score = desired_grade - pred_grade[0] # rating based on how close it can get to 4.0

    return score

res = gp_minimize(objective, space, n_calls=50, random_state=0)
optimized_habits = { #param not to be optimized
    'Age': age,
    'Gender': gender,
    'ParentalEducation': parental_education
}
optimized_habits.update(dict(zip([dim.name for dim in space], res.x)))

print("Best score: ", res.fun)
print("Best parameters: ", res.x)

   Age  Gender  ParentalEducation  StudyTimeWeekly  Absences  Tutoring  \
0   15       0                  1        11.856892        24         1   

   ParentalSupport  Extracurricular  Sports  Music  Volunteering  
0                3                1       0      0             0  
[1.3533225]
   Age  Gender  ParentalEducation  StudyTimeWeekly  Absences  Tutoring  \
0   15       0                  1         5.453126        14         1   

   ParentalSupport  Extracurricular  Sports  Music  Volunteering  
0                2                0       1      0             1  
[2.0084481]
   Age  Gender  ParentalEducation  StudyTimeWeekly  Absences  Tutoring  \
0   15       0                  1         7.364831        28         0   

   ParentalSupport  Extracurricular  Sports  Music  Volunteering  
0                3                0       1      1             1  
[0.67229664]
   Age  Gender  ParentalEducation  StudyTimeWeekly  Absences  Tutoring  \
0   15       0                  1       

# KNN Examples

In [18]:
# Create the neighborhood DataFrame by dropping specific columns
neighborhood = data.drop(columns=['StudyTimeWeekly', 'Absences',
                                   'Tutoring', 'ParentalSupport', 'Extracurricular', 'Sports', 'Music',
                                   'Volunteering'])
print(neighborhood.head())

user_query = {
    'Age': age,
    'Gender': gender,
    'ParentalEducation': parental_education,
    'GPA': desired_grade  # simulate that we want to be near those with target GPA
}
user_query = pd.DataFrame(user_query, index=[0])

# apply weights
weights = {
    'Age': 1.0,
    'Gender': 2.0,
    'ParentalEducation': 1.0,
    'GPA': 100.0
}

weighted_neighborhood = neighborhood.copy()
weighted_user_query = user_query.copy()

for feature, weight in weights.items():
    weighted_neighborhood[feature] *= weight
    weighted_user_query[feature] *= weight

nbrs_norm = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(weighted_neighborhood)
distances3, indices3 = nbrs_norm.kneighbors(weighted_user_query)

nearest_neighbors = data.iloc[indices3.flatten()]
print("Similar successful students:\n", nearest_neighbors[['GPA', 'StudyTimeWeekly', 'Absences', 'Extracurricular', 'Sports', 'Music', 'Volunteering', 'Tutoring']])


   Age  Gender  ParentalEducation       GPA
0   17       1                  2  2.929196
1   18       0                  1  3.042915
2   15       0                  3  0.112602
3   17       1                  3  2.054218
4   17       1                  2  1.288061
Similar successful students:
       GPA  StudyTimeWeekly  Absences  Extracurricular  Sports  Music  \
1919  4.0        17.442121         1                1       1      0   
442   4.0        19.424398         0                0       1      1   
1278  4.0        18.899696         3                1       1      0   
2319  4.0         9.285447         0                1       0      1   
2028  4.0        18.656924         0                1       0      0   

      Volunteering  Tutoring  
1919             0         1  
442              1         0  
1278             0         1  
2319             1         0  
2028             1         1  


# GPT For Text Generation

In [28]:
openai.api_key = getpass.getpass("🔐 Enter your OpenAI API key: ")

🔐 Enter your OpenAI API key: ··········


In [29]:
# just an exmaple
current_habits = {
    'StudyTimeWeekly': study_time_weekly,
    'Absences': absences,
    'Extracurricular': extracurricular,
    'Sports': sports,
    'Music': music,
    'Volunteering': volunteering,
    'ParentalSupport': parental_support,
    'ParentalEducation': parental_education,
    'Tutoring': tutoring
}


def generate_recommendations_gpt4(current_habits, optimized_habits, desired_grade):
    prompt = f"""
Student is aiming to improve GPA to {desired_grade}.

Current Habits:
- Study Time: {current_habits['StudyTimeWeekly']} hrs
- Absences: {current_habits['Absences']}
- Extracurricular: {current_habits['Extracurricular']}
- Sports: {current_habits['Sports']}
- Music: {current_habits['Music']}
- Volunteering: {current_habits['Volunteering']}
- Parental Support Level: {current_habits['ParentalSupport']}
- Parental Education Level: {current_habits['ParentalEducation']}
- Tutoring: {current_habits['Tutoring']}

Recommended Optimized Habits:
- Study Time: {optimized_habits['StudyTimeWeekly']} hrs
- Absences: {optimized_habits['Absences']}
- Extracurricular: {optimized_habits['Extracurricular']}
- Sports: {optimized_habits['Sports']}
- Music: {optimized_habits['Music']}
- Volunteering: {optimized_habits['Volunteering']}
- Parental Support Level: {optimized_habits['ParentalSupport']}
- Parental Education Level: {optimized_habits['ParentalEducation']}
- Tutoring: {optimized_habits['Tutoring']}

As an academic advisor, explain how these changes will help improve the student’s GPA. Also provide personalized and motivational advice to help them stay on track.
"""

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=600
    )

    return response.choices[0].message.content


In [30]:
recommendation_text = generate_recommendations_gpt4(current_habits, optimized_habits, desired_grade)
print(recommendation_text)


The recommended changes will significantly help in improving your GPA. Increasing your study time from 4.2 hours to 20 hours will provide you with more time to understand the material, complete assignments, and prepare for exams. This will not only help you perform better in tests but also build a solid foundation of the material for future reference. 

Reducing absences to zero is crucial because each class is an opportunity to learn from the teacher directly, interact with classmates, and participate in discussions. This can greatly enhance your understanding of the material and improve your grades. 

Participating in sports and music can actually help improve your academic performance. These activities require discipline, time management, teamwork, and focus, all of which are transferable skills to your academics. They also provide a necessary break from studying, helping you maintain balance and reducing the risk of burnout.

A high level of parental support can provide motivation 

# Demo using Gradio

In [33]:
import gradio as gr
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def demo_app(age, gender, parental_education, study_time, absences, tutoring, parental_support,
             extracurricular, sports, music, volunteering, target_gpa):

    current_habits = {
        'Age': age,
        'Gender': gender,
        'ParentalEducation': parental_education,
        'StudyTimeWeekly': study_time,
        'Absences': absences,
        'Tutoring': tutoring,
        'ParentalSupport': parental_support,
        'Extracurricular': extracurricular,
        'Sports': sports,
        'Music': music,
        'Volunteering': volunteering
    }

    def optimize(user_fixed):
        @use_named_args(space)
        def objective(**params):
            user_data = user_fixed.copy()
            user_data.update(params)
            df = pd.DataFrame(user_data, index=[0])
            input_vec = scaler.transform(df)
            pred = reg.predict(input_vec)[0]
            return abs(target_gpa - pred)

        result = gp_minimize(objective, space, n_calls=50, random_state=0)
        return dict(zip([dim.name for dim in space], result.x))

    user_fixed = {
        'Age': age,
        'Gender': gender,
        'ParentalEducation': parental_education
    }
    optimized_values = optimize(user_fixed)

    optimized_habits = {
        'StudyTimeWeekly': optimized_values['StudyTimeWeekly'],
        'Absences': optimized_values['Absences'],
        'Tutoring': optimized_values['Tutoring'],
        'ParentalSupport': optimized_values['ParentalSupport'],
        'Extracurricular': optimized_values['Extracurricular'],
        'Sports': optimized_values['Sports'],
        'Music': optimized_values['Music'],
        'Volunteering': optimized_values['Volunteering']
    }

    summary = generate_recommendations_gpt4(current_habits, {**user_fixed, **optimized_habits}, target_gpa)

    query = {
        'Age': age,
        'Gender': gender,
        'ParentalEducation': parental_education,
        'GPA': target_gpa
    }
    user_query_df = pd.DataFrame(query, index=[0])
    weighted_user_query = user_query_df.copy()
    weighted_neighborhood = neighborhood.copy()

    for feat, w in weights.items():
        weighted_user_query[feat] *= w
        weighted_neighborhood[feat] *= w

    knn = NearestNeighbors(n_neighbors=5)
    knn.fit(weighted_neighborhood)
    _, indices = knn.kneighbors(weighted_user_query)

    similar_students = data.iloc[indices.flatten()]
    example_table = similar_students[[
        'GPA', 'StudyTimeWeekly', 'Absences', 'Extracurricular', 'Sports', 'Music', 'Volunteering', 'Tutoring'
    ]]

    result_table = pd.DataFrame([optimized_habits])
    return result_table, example_table, summary

# the gradio interface
def app_ui():
    with gr.Blocks() as app:
        gr.Markdown("### 🧠 Fill in your current habits and target GPA:")

        with gr.Row():
            age = gr.Number(label="🧒 Age", value=15)
            gender = gr.Radio([0, 1], label="⚧️ Gender (0=Male, 1=Female)", value=0)
            parental_education = gr.Dropdown(
                choices=[0, 1, 2, 3, 4],
                label="🎓 Parental Education (0=None, 1=High School, 2=College, 3=Bachelor's, 4=Higher)",
                value=1
            )

        with gr.Row():
            study_time = gr.Number(label="📘 Study Time Weekly (hrs)", value=0.0)
            absences = gr.Number(label="🚫 Absences (0–30)", value=10)
            tutoring = gr.Radio([0, 1], label="🎓 Tutoring (0=No, 1=Yes)", value=0)
            parental_support = gr.Slider(0, 4, step=1,
                label="👨‍👩‍👧 Parental Support (0=None to 4=Very High)", value=0)

        with gr.Row():
            extracurricular = gr.Radio([0, 1], label="🎭 Extracurricular", value=0)
            sports = gr.Radio([0, 1], label="🏀 Sports", value=0)
            music = gr.Radio([0, 1], label="🎵 Music", value=0)
            volunteering = gr.Radio([0, 1], label="🙌 Volunteering", value=0)

        target_gpa = gr.Number(label="🎯 Target GPA", value=0.0)

        with gr.Row():
            submit = gr.Button("🚀 Get Personalized Plan")

        output1 = gr.Dataframe(label="✅ Optimized Study Habits")
        output2 = gr.Dataframe(label="📊 Similar Students (KNN)")
        output3 = gr.Textbox(label="🧠 GPT Summary", lines=8)

        submit.click(fn=demo_app,
                     inputs=[age, gender, parental_education,
                             study_time, absences, tutoring, parental_support,
                             extracurricular, sports, music, volunteering, target_gpa],
                     outputs=[output1, output2, output3])
    return app

#Our Landing Page
with gr.Blocks() as landing:
    gr.Markdown("""
    <center>
    <h1>🎓 <span style='color:#4A90E2'>Your Personalized GPA Booster:</span> SmartStudy</h1>
    <h3>By <i>Study Architects</i></h3>
    <p style="max-width: 700px; font-size: 17px;">
    A smart tool that helps students reach their academic goals by recommending better study habits,
    backed by real student data and explained using GPT-4.
    </p>
    </center>
    """)

    start_btn = gr.Button("✨ Get Started")
    app_container = gr.Column(visible=False)

    start_btn.click(fn=lambda: gr.update(visible=True), outputs=app_container)

    with app_container:
        app_ui()

landing.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://197474a425f12350a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
